In [5]:
import os, sys
import pandas as pd
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import importlib
import utils.event_extractor  # your module

importlib.reload(utils.event_extractor)
from utils.event_extractor import EventExtractor  # re-import your class if needed


filtered_reports_df = pd.read_pickle("../exports/filtered_patient_reports.pkl")
extractor = EventExtractor(event_name_model_type="biolord", attribute_model_type="llama3")

def extract_events(sentences):
    global extractor
    event_types = ["Pain", "Sleep"]
    events = extractor.extract_events(sentences=sentences, event_names=event_types, threshold=0.2)
    return events


In [ ]:
import glob, os
os.makedirs("../exports/filtered_patient_reports_with_event_log/",exist_ok=True)
batch_size = 500
for i in range(0, len(filtered_reports_df), batch_size):
    batch = filtered_reports_df.iloc[i:i+batch_size]
    batch["Events"] = batch['Sentences'].apply(extract_events)
    batch.to_pickle(f"../exports/filtered_patient_reports_with_event_log/batch_{i//batch_size}.pkl")
batch_files = sorted(glob.glob("../exports/filtered_patient_reports_with_event_log/batch_*.pkl"))
combined_df = pd.concat([pd.read_pickle(f) for f in batch_files], ignore_index=True)
combined_df.to_pickle("../exports/filtered_patient_reports_with_event_log/combined.pkl")